In [1]:
import z3

We don't need to solve for all of the trajectory intersections at once; picking just a handful will do.  

There are initially six unknowns (the three components each of $\overrightarrow{x_{rock}}$ and $\overrightarrow{v_{rock}}$), and each additional equation adds three equations of the form $x_{i\cdot rock} + v_{i \cdot rock} t_n = x_{i\cdot hail\cdot n} + v_{i \cdot hail\cdot n} t_n$, which also contains one new unknown, the time of intersection $t_n$.

| Hail | Equations | Unknowns |
| ---- | ----------- | ----------- |
| 1 | 3 | 7 |
| 2 | 6 | 8 |
| 3 | 9 | 9 |

We just need 3 hail trajectories.  The first three in the set are enough.

In [2]:
hail = [[int(i) for i in l.replace('@',',').split(',')]
                for l in open('input')]

hail_sample = hail[0:3]

hail_position = [h[0:3] for h in hail_sample]
hail_velocity = [h[3:] for h in hail_sample]

hail_sample

[[156689809620606, 243565579389165, 455137247320393, -26, 48, -140],
 [106355761063908, 459832650718033, 351953299411025, 73, -206, -52],
 [271915251832336, 487490927073225, 398003502953444, 31, -414, -304]]

In [3]:
start_position = z3.RealVector('p', 3)
start_velocity = z3.RealVector('v', 3)
hit_time = z3.RealVector('t', 3)

For each of the hailstone trajectories in the sample, the thrown rock's coordinates will be equal to the hailstone's coordinates at a given time $t$: $x_{i\cdot rock} + v_{i \cdot rock} t = x_{i\cdot hail} + v_{i \cdot hail} t$ for $i \in \{x,\,y,\,z\}$.

In [4]:
solver = z3.Solver()
solver.add(*[start_position[i] + start_velocity[i] * t == hp[i] + hv[i] * t
             for t, hp, hv in zip(hit_time, hail_position, hail_velocity) for i in range(3)])

Is the solver satisfied?  That is, do none of the constraints conflict with each other?

In [5]:
solver.check()

sat

The solver model returns the state that satisfies all the constraints:

In [6]:
solver.model()

[t__2 = 474506740599,
 t__0 = 931974028142,
 v__0 = -337,
 t__1 = 829702369046,
 v__1 = -6,
 v__2 = 155,
 p__0 = 446533732372768,
 p__2 = 180204909018503,
 p__1 = 293892176908833]

and all that's left to do is extract the results from the state and calculate the answer.

In [7]:
solver.model().eval(sum(start_position))

920630818300104